<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/nmt_with_attention_KA_lemmatization_application_v04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Preparing data

## TreeTagger - Georgian

In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2

wget https://heibox.uni-heidelberg.de/f/a6f7f36f175942ccad0a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian
chmod a+x /content/treetagger/cmd/tree-tagger-georgian


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
!mv index.html?dl=1 georgianrandom02.txt

In [ ]:
!wc georgianrandom02.txt

In [4]:
!./treetagger/cmd/tree-tagger-georgian georgianrandom02.txt >georgianrandom02.vert

	reading parameters ...
	tagging ...
11301000	 finished.


In [5]:
!wc georgianrandom02.vert

 11301824  33112747 394150707 georgianrandom02.vert


## working with complete specialized Georgian corpus

In [6]:
import os, sys, re

def removeNewLines(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')

    for SLine in FIn:
        SLine = SLine.strip()
        SLine = re.sub('- ', '', SLine)
        if SLine == '': 
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            continue

        FOut.write(SLine + ' ')
    FOut.flush()
    return


!wget https://heibox.uni-heidelberg.de/f/dc5bcb4413aa42668130/?dl=1
!mv index.html?dl=1 specialisedCorpus.zip
!unzip specialisedCorpus.zip
!wc specialised-corpora/*
!mkdir specialised-corpora-s01-lines/

removeNewLines('specialised-corpora/cFiktion.txt', 'specialised-corpora-s01-lines/cFiktion.txt')
removeNewLines('specialised-corpora/cNaturwissenschaft.txt', 'specialised-corpora-s01-lines/cNaturwissenschaft.txt')
removeNewLines('specialised-corpora/cRechtswissenschaft.txt', 'specialised-corpora-s01-lines/cRechtswissenschaft.txt')

!mkdir specialised-corpora-s02-vert/

!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cFiktion.txt >specialised-corpora-s02-vert/cFiktion.vert
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cNaturwissenschaft.txt >specialised-corpora-s02-vert/cNaturwissenschaft.vert
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cRechtswissenschaft.txt >specialised-corpora-s02-vert/cRechtswissenschaft.vert


--2023-06-09 12:20:00--  https://heibox.uni-heidelberg.de/f/dc5bcb4413aa42668130/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/82b826cd-d45a-4b29-ad72-b4c0416f3ea1/specialised-corpora.zip [following]
--2023-06-09 12:20:01--  https://heibox.uni-heidelberg.de/seafhttp/files/82b826cd-d45a-4b29-ad72-b4c0416f3ea1/specialised-corpora.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 8265764 (7.9M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]   7.88M  4.84MB/s    in 1.6s    

2023-06-09 12:20:03 (4.84 MB/s) - ‘index.html?dl=1’ saved [8265764/8265764]

Archive:  specialisedCorpus.zip
   creating: specialised-corpora/
  inflating: spec

In [7]:
# how many words in each corpus
!wc specialised-corpora-s02-vert/*

  743107  2229321 24381882 specialised-corpora-s02-vert/cFiktion.vert
  638176  1914550 22999131 specialised-corpora-s02-vert/cNaturwissenschaft.vert
 1141877  3425631 43921072 specialised-corpora-s02-vert/cRechtswissenschaft.vert
 2523160  7569502 91302085 total


In [8]:
!mv /content/georgianrandom02.vert /content/georgianrandom01.vert

In [9]:
!cat /content/georgianrandom01.vert /content/specialised-corpora-s02-vert/cFiktion.vert /content/specialised-corpora-s02-vert/cNaturwissenschaft.vert /content/specialised-corpora-s02-vert/cRechtswissenschaft.vert >/content/georgianrandom02.vert

In [10]:
!wc georgianrandom01.vert
!wc georgianrandom02.vert

 11301824  33112747 394150707 georgianrandom01.vert
 13824984  40682249 485452792 georgianrandom02.vert


## selecting unknown elements

In [11]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-unknown.txt
!awk -F '\t' '($3!="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-known.txt

2581616
11243368


### finding words with letters only

In [12]:
!awk -F '\t' '($3 ~ /[Ⴀ-ჼ]/){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-ka-alphabet.txt

7931513


In [13]:
!awk -F '\t' '($1 != $3){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02-ka-alphabet.txt >georgianrandom02-ka-change-wordf2lem.txt

3364030


In [14]:
!wc georgianrandom02-ka-alphabet.txt
!head -40 georgianrandom02-ka-alphabet.txt

!wc georgianrandom02-ka-change-wordf2lem.txt
!head -40 georgianrandom02-ka-change-wordf2lem.txt

  7931513  23794539 357780572 georgianrandom02-ka-alphabet.txt
შორ	JSU	შორი
მანძილზე	NSD:II	მანძილი
გადამცემის	NSG	გადამცემა
იდეა	NSN	იდეა
უძველესი	JSN	უძველესი
დროიდან	NSI:II	დრო
არსებობდა,თუმცა	NSN	არსებობდა,თუმცა
ასეთი	PDSN	ასეთი
შექმნის	V:3S:F	შექმნის
ტექნიკური	JSN	ტექნიკური
და	CC	და
თეორიული	JSN	თეორიული
საფუძველი	NSN	საფუძველი
მხოლოდ	RR	მხოლოდ
საუკუნის	NSG	საუკუნე
გამოჩნდა	V:3S:A	გამოჩნდება
მას	PDSD	იგი
შემდეგ	RR	შემდეგ
რაცა	PTSN:XO	რა
ადამიანმა	NSE	ადამიანი
დიდი	JSN	დიდი
და	CC	და
შრომის	NSG	შრომა
შედეგად	NSA	შედეგი
გადაცემა	NSN	გადაცემა
მოახერხა	V:3S:A	მოახერხებს
ტელევიზორი	NSN	ტელევიზორი
ეს	PDSU	ეს
არის	V:3S:P	არის
ტექნოლოგია	NSN	ტექნოლოგია
რომელიც	PTSN:XO	რომელი
საშუალებას	NSD	საშუალება
გვაძლევს	V:3S1P:P	აძლევს
შორ	JSU	შორი
მანძილზე	NSD:II	მანძილი
მოძრავი	JSN	მოძრავი
ხმის	NSG	ხმა 
სიტყვა	NSN	სიტყვა
ქართულად	JSA	ქართული
შორს	RR	შორს
  3364030  10092090 165307657 georgianrandom02-ka-change-wordf2lem.txt
შორ	JSU	შორი
მანძილზე	NSD:II	მანძილი
გადამცემის	NSG	გადამცემა
დროიდან	NSI:II

In [15]:
def corp2frqDict(SFIn):
    DFrq = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip()
            # LLine = SLine.split('\t')
            try:
                DFrq[SLine]+=1
            except:
                DFrq[SLine]=1
    return DFrq


def printFrqDict(DFrq, SFOut):
    FOut = open(SFOut, 'w')
    count = 0
    for key, val in sorted(DFrq.items(), key=lambda item: item[1], reverse=True):
        count+=1
        FOut.write(f'{count}\t{key}\t{val}\n')
    FOut.flush()

In [ ]:
DFrqKnown = corp2frqDict('georgianrandom02-known.txt')
print(len(DFrqKnown))
printFrqDict(DFrqKnown, 'georgianrandom02-known-frq.txt')

DFrqUnKnown = corp2frqDict('georgianrandom02-unknown.txt')
print(len(DFrqUnKnown))
printFrqDict(DFrqUnKnown, 'georgianrandom02-unknown-frq.txt')

DFrqUnKnown = corp2frqDict('georgianrandom02-ka-alphabet.txt')
print(len(DFrqUnKnown))
printFrqDict(DFrqUnKnown, 'georgianrandom02-ka-alphabet-frq.txt')

DFrqUnKnown = corp2frqDict('georgianrandom02-ka-change-wordf2lem.txt')
print(len(DFrqUnKnown))
printFrqDict(DFrqUnKnown, 'georgianrandom02-ka-change-wordf2lem.txt')

In [ ]:
%%bash
wc georgianrandom02-known-frq.txt
head -n 40 georgianrandom02-known-frq.txt

wc georgianrandom02-unknown-frq.txt
head -n 40 georgianrandom02-unknown-frq.txt

wc georgianrandom02-ka-alphabet-frq.txt
head -n 40 georgianrandom02-ka-alphabet-frq.txt

wc georgianrandom02-ka-change-wordf2lem.txt
head -n 40 georgianrandom02-ka-change-wordf2lem.txt

## Tasks to further refine the training set:
- For known words: find base forms, which do not change during lemmatization (by comparing them to ka-change-word2lem file, which ones occur in both...)
- Idenfiy parts-of-speech which do not change
- lemmatize those parts-of-speech to the word token using the PoS code only
- Experiment with using the PoS code as an additional factorin S2S model


In [ ]:
!awk -F '\t' '{j++; printf "%s\t%s\n", $2, $4} END{print j+0  > "/dev/stderr" }' <georgianrandom02-ka-change-wordf2lem.txt >georgianrandom02-ka-change-set.txt
!awk -F '\t' '{j++; printf "%s_%s\t%s\n", $2, $3, $4} END{print j+0  > "/dev/stderr" }' <georgianrandom02-ka-change-wordf2lem.txt >georgianrandom02-ka-change-pos-set.txt

In [ ]:
!echo georgianrandom02-ka-change-set.txt
!head -n 20 georgianrandom02-ka-change-set.txt
!echo georgianrandom02-ka-change-pos-set.txt
!head -n 20 georgianrandom02-ka-change-pos-set.txt

In [ ]:
Word = 'texting'
Word = ' '.join(list(Word))
print(Word)

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/70af69f479044d228e6d/?dl=1
!mv index.html?dl=1 ltKA.txt

In [ ]:
def readTableWS(SFIn):
    DMap = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip('\n')
            try:
                KA, LA = SLine.split('\t')
                DMap[KA] = LA
            except:
                continue
    print(len(DMap))
    return DMap

In [ ]:
DTransliterationKA = readTableWS('/content/ltKA.txt')

In [ ]:
S2Print = ''.join([DTransliterationKA[x] for x in 'ქართული' if x in DTransliterationKA])

In [ ]:
print(S2Print)

In [ ]:
import os, sys, re
def splitChars(SFIn, SFOut, BSplitChars = True, RandomEvery = 10, DTranslit = None):
    FOut = open(SFOut, 'w')
    FOutTest = open(SFOut + '_test.txt', 'w')
    
    with open(SFIn) as FIn:
        countW = 0
        for SLine in FIn:
            countW += 1
            SLine = SLine.rstrip()
            LLine = SLine.split('\t')
            try:
                LHS = LLine[0]
                RHS = LLine[1]
            except:
                LHS = ''
                RHS = ''
                sys.stderr.write('.')
                continue
            try:
                LWordPoS = LHS.split('_')
                SWord = LWordPoS[0]
                SPoS = LWordPoS[1]

            except:
                SWord = LHS
                SPoS = None

            if BSplitChars:
                SWord = ' '.join(list(SWord))
                RHS = ' '.join(list(RHS))


            if DTranslit:
                SWord = ''.join([DTransliterationKA[x] for x in SWord if x in DTranslit])
                RHS = ''.join([DTransliterationKA[x] for x in RHS if x in DTranslit])
                

            if RandomEvery and countW % RandomEvery == 0:
                if SPoS: 
                    FOutTest.write(f'{SWord} {SPoS}\t{RHS}\n')
                else:
                    FOutTest.write(f'{SWord}\t{RHS}\n')
                continue
                

            if SPoS: 
                FOut.write(f'{SWord} {SPoS}\t{RHS}\n')
            else:
                FOut.write(f'{SWord}\t{RHS}\n')

    return

In [ ]:
splitChars('georgianrandom02-ka-change-set.txt', 'georgianrandom02-ka-change-chars.txt', BSplitChars = True, RandomEvery = 500, DTranslit = DTransliterationKA)
splitChars('georgianrandom02-ka-change-pos-set.txt', 'georgianrandom02-ka-change-pos-chars.txt', BSplitChars = True, RandomEvery = 500, DTranslit = DTransliterationKA)

In [ ]:
!echo input: georgianrandom02-ka-change-chars.txt
!wc georgianrandom02-ka-change-chars.txt
!head -n 40 georgianrandom02-ka-change-chars.txt
!echo output: georgianrandom02-ka-change-pos-chars.txt
!wc georgianrandom02-ka-change-pos-chars.txt georgianrandom02-ka-change-pos-chars.txt_test.txt
!echo 
!echo georgianrandom02-ka-change-pos-chars.txt
!head -40 georgianrandom02-ka-change-pos-chars.txt
!wc georgianrandom02-ka-change-pos-chars.txt
!wc georgianrandom02-ka-change-pos-chars.txt_test.txt


# Neural machine translation with attention

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/nmt_with_attention">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/nmt_with_attention.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to train a sequence-to-sequence (seq2seq) model for Spanish-to-English translation roughly based on [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025v5) (Luong et al., 2015). 

<table>
<tr>
  <td>
   <img width=400 src="https://www.tensorflow.org/images/tutorials/transformer/RNN%2Battention-words-spa.png"/>
  </td>
</tr>
<tr>
  <th colspan=1>This tutorial: An encoder/decoder connected by attention.</th>
<tr>
</table>

While this architecture is somewhat outdated, it is still a very useful project to work through to get a deeper understanding of sequence-to-sequence models and attention mechanisms (before going on to [Transformers](transformer.ipynb)).



This example assumes some knowledge of TensorFlow fundamentals below the level of a Keras layer:
  * [Working with tensors](https://www.tensorflow.org/guide/tensor) directly
  * [Writing custom `keras.Model`s and `keras.layers`](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

After training the model in this notebook, you will be able to input a Spanish sentence, such as "*¿todavia estan en casa?*", and return the English translation: "*are you still at home?*"

The resulting model is exportable as a `tf.saved_model`, so it can be used in other TensorFlow environments.

The translation quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while translating:

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot">

Note: This example takes approximately 10 minutes to run.

## Setup

In [ ]:
!pip install "tensorflow-text>=2.11"
!pip install einops

In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

This tutorial uses a lot of low level API's where it's easy to get shapes wrong. This class is used to check shapes throughout the tutorial.


In [ ]:
#@title
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

## The data

The tutorial uses a language dataset provided by [Anki](http://www.manythings.org/anki/). This dataset contains language translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

They have a variety of languages available, but this example uses the English-Spanish dataset.

### Download and prepare the dataset

For convenience, a copy of this dataset is hosted on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps you need to take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [ ]:
# Download the file
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

In [ ]:
# OR ::
import pathlib

In [ ]:
## OR
## load georgian dataset:
# Download the file
path_to_file = '/content/georgianrandom02-ka-change-chars.txt'

In [ ]:
## OR
## load georgian dataset with pos codes:
# Download the file
path_to_file = '/content/georgianrandom02-ka-change-pos-chars.txt'

In [ ]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target, context in pairs])
  target = np.array([target for target, context in pairs])

  return target, context

In [ ]:
def load_data_ch(path):
  pairs = []
  with open(path) as FIn:
    countW = 0
    for SLine in FIn:
      SLine = SLine.strip()
      try:
        ctx, trg = SLine.split('\t')
      except:
        sys.stderr.write('.')
        continue
      pairs.append((trg, ctx))
    
    context = np.array([context for target, context in pairs])
    target = np.array([target for target, context in pairs])

  return target, context    

In [ ]:
target_raw, context_raw = load_data_ch(path_to_file)
print(context_raw[-1])
print(context_raw[-2])
print(context_raw[99])

In [ ]:
# target_raw, context_raw = load_data(path_to_file)
print(context_raw[-1])

In [ ]:
print(target_raw[-1])

In [ ]:
print(len(context_raw))
type(target_raw)


In [ ]:
print(len(context_raw))
type(context_raw)

### Create a tf.data dataset

From these arrays of strings you can create a `tf.data.Dataset` of strings that shuffles and batches them efficiently:

In [ ]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [ ]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

### Text preprocessing

One of the goals of this tutorial is to build a model that can be exported as a `tf.saved_model`. To make that exported model useful it should take `tf.string` inputs, and return `tf.string` outputs: All the text processing happens inside the model. Mainly using a `layers.TextVectorization` layer.

#### Standardization

The model is dealing with multilingual text with a limited vocabulary. So it will be important to standardize the input text.

The first step is Unicode normalization to split accented characters and replace compatibility characters with their ASCII equivalents.

The `tensorflow_text` package contains a unicode normalize operation:

In [ ]:
example_text = tf.constant('¿Todavía está en casa?')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

In [ ]:
example_text = tf.constant('რომლებმაც')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

Unicode normalization will be the first step in the text standardization function:

In [ ]:
def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [ ]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

#### Text Vectorization

This standardization function will be wrapped up in a `tf.keras.layers.TextVectorization` layer which will handle the vocabulary extraction and conversion of input text to sequences of tokens.

In [ ]:
max_vocab_size = 500

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

The `TextVectorization` layer and many other [Keras preprocessing layers](https://www.tensorflow.org/guide/keras/preprocessing_layers) have an `adapt` method. This method reads one epoch of the training data, and works a lot like `Model.fit`. This `adapt` method initializes the layer based on the data. Here it determines the vocabulary:

In [ ]:
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:40]

That's the Spanish `TextVectorization` layer, now build and `.adapt()` the English one:

In [ ]:
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:40]

Now these layers can convert a batch of strings into a batch of token IDs:

In [ ]:
example_tokens = context_text_processor(example_context_strings)
example_tokens[:3, :]

The `get_vocabulary` method can be used to convert token IDs back to text:

In [ ]:
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]
' '.join(tokens)

The returned token IDs are zero-padded. This can easily be turned into a mask:

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(example_tokens.to_tensor())
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(example_tokens.to_tensor() != 0)
plt.title('Mask')

### Process the dataset



The `process_text` function below converts the `Datasets` of strings, into  0-padded tensors of token IDs. It also converts from a `(context, target)` pair to an `((context, target_in), target_out)` pair for training with `keras.Model.fit`. Keras expects `(inputs, labels)` pairs, the inputs are the `(context, target_in)` and the labels are `target_out`. The difference between `target_in` and `target_out` is that they are shifted by one step relative to eachother, so that at each location the label is the next token.

In [ ]:
def process_text(context, target):
  context = context_text_processor(context).to_tensor()
  target = target_text_processor(target)
  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  return (context, targ_in), targ_out


train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

Here is the first sequence of each, from the first batch:

In [ ]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
  print(ex_context_tok[0, :10].numpy()) 
  print()
  print(ex_tar_in[0, :10].numpy()) 
  print(ex_tar_out[0, :10].numpy()) 

## The encoder/decoder

The following diagrams shows an overview of the model. In both the encoder is on the left, the decoder is on the right. At each time-step the decoder's output is combined with the encoder's output, to predict the next word. 

The original [left] contains a few extra connections that are intentionally omitted from this tutorial's model [right], as they are generally unnecessary, and difficult to implement. Those missing connections are:

1. Feeding the state from the encoder's RNN to the decoder's RNN
2. Feeding the attention output back to the RNN's input.

<table>
<tr>
  <td>
   <img width=500 src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg"/>
  </td>
  <td>
   <img width=380 src="https://www.tensorflow.org/images/tutorials/transformer/RNN+attention.png"/>
  </td>
</tr>
<tr>
  <th colspan=1>The original from <a href=https://arxiv.org/abs/1508.04025v5>Effective Approaches to Attention-based Neural Machine Translation</a></th>
  <th colspan=1>This tutorial's model</th>
<tr>
</table>


Before getting into it define constants for the model:

In [ ]:
UNITS = 256

### The encoder

The goal of the encoder is to process the context sequence into a sequence of vectors that are useful for the decoder as it attempts to predict the next output for each timestep. Since the context sequence is constant, there is no restriction on how information can flow in the encoder, so use a bidirectional-RNN to do the processing:

<table>
<tr>
  <td>
   <img width=500 src="https://tensorflow.org/images/tutorials/transformer/RNN-bidirectional.png"/>
  </td>
</tr>
<tr>
  <th>A bidirectional RNN</th>
<tr>
</table>

The encoder:

1. Takes a list of token IDs (from `context_text_processor`).
3. Looks up an embedding vector for each token (Using a `layers.Embedding`).
4. Processes the embeddings into a new sequence (Using a bidirectional `layers.GRU`).
5. Returns the processed sequence. This will be passed to the attention head.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, text_processor, units):
    super(Encoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units
    
    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, units,
                                               mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, x):
    shape_checker = ShapeChecker()
    shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x = self.embedding(x)
    shape_checker(x, 'batch s units')

    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    shape_checker(x, 'batch s units')

    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context

Try it out:

In [ ]:
# Encode the input sequence.
encoder = Encoder(context_text_processor, UNITS)
ex_context = encoder(ex_context_tok)

print(f'Context tokens, shape (batch, s): {ex_context_tok.shape}')
print(f'Encoder output, shape (batch, s, units): {ex_context.shape}')

### The attention layer

The attention layer lets the decoder access the information extracted by the encoder. It computes a vector from the entire context sequence, and adds that to the decoder's output. 

The simplest way you could calculate a single vector from the entire sequence would be to take the average across the sequence (`layers.GlobalAveragePooling1D`). An attention layer is similar, but calculates a **weighted** average across the context sequence. Where the weights are calculated from the combination of context and "query" vectors.

<table>
<tr>
  <td>
   <img width=500 src="https://www.tensorflow.org/images/tutorials/transformer/CrossAttention-new-full.png"/>
  </td>
</tr>
<tr>
  <th colspan=1>The attention layer</th>
<tr>
</table>

In [ ]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    shape_checker = ShapeChecker()
 
    shape_checker(x, 'batch t units')
    shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)
    
    shape_checker(x, 'batch t units')
    shape_checker(attn_scores, 'batch heads t s')
    
    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
attention_layer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(target_text_processor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)
ex_tar_embed = embed(ex_tar_in)

result = attention_layer(ex_tar_embed, ex_context)

print(f'Context sequence, shape (batch, s, units): {ex_context.shape}')
print(f'Target sequence, shape (batch, t, units): {ex_tar_embed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attention_layer.last_attention_weights.shape}')

The attention weights will sum to `1` over the context sequence, at each location in the target sequence.

In [ ]:
attention_layer.last_attention_weights[0].numpy().sum(axis=-1)



Here are the attention weights across the context sequences at `t=0`:

In [ ]:
attention_weights = attention_layer.last_attention_weights
mask=(ex_context_tok != 0).numpy()

plt.subplot(1, 2, 1)
plt.pcolormesh(mask*attention_weights[:, 0, :])
plt.title('Attention weights')

plt.subplot(1, 2, 2)
plt.pcolormesh(mask)
plt.title('Mask');


Because of the small-random initialization the attention weights are initially all close to `1/(sequence_length)`. The model will learn to make these less uniform as training progresses.

### The decoder

The decoder's job is to generate predictions for the next token at each location in the target sequence.

1. It looks up embeddings for each token in the target sequence.
2. It uses an RNN to process the target sequence, and keep track of what it has generated so far.
3. It uses RNN output as the "query" to the attention layer, when attending to the encoder's output.
4. At each location in the output it predicts the next token.

When training, the model predicts the next word at each location. So it's important that the information only flows in one direction through the model. The decoder uses a unidirectional (not bidirectional) RNN to process the target sequence.

When running inference with this model it produces one word at a time, and those are fed back into the model.

<table>
<tr>
  <td>
   <img width=500 src="https://tensorflow.org/images/tutorials/transformer/RNN.png"/>
  </td>
</tr>
<tr>
  <th>A unidirectional RNN</th>
<tr>
</table>

Here is the `Decoder` class' initializer. The initializer creates all the necessary layers.

In [ ]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

#### Training

Next, the `call` method, takes 3 arguments:

* `inputs` -  a `context, x` pair where:
  * `context` - is the context from the encoder's output.
  * `x` - is the target sequence input.
* `state` - Optional, the previous `state` output from the decoder (the internal state of the decoder's RNN). Pass the state from a previous run to continue generating text where you left off.
* `return_state` - [Default: False] - Set this to `True` to return the RNN state. 

In [ ]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  shape_checker = ShapeChecker()
  shape_checker(x, 'batch t')
  shape_checker(context, 'batch s units')

  # 1. Lookup the embeddings
  x = self.embedding(x)
  shape_checker(x, 'batch t units')

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  shape_checker(x, 'batch t units')

  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights
  shape_checker(x, 'batch t units')
  shape_checker(self.last_attention_weights, 'batch t s')

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)
  shape_checker(logits, 'batch t target_vocab_size')

  if return_state:
    return logits, state
  else:
    return logits

That will be sufficient for training. Create an instance of the decoder to test out:

In [ ]:
decoder = Decoder(target_text_processor, UNITS)

In training you'll use the decoder like this:

Given the context and target tokens, for each target token it predicts the next target token. 

In [ ]:
logits = decoder(ex_context, ex_tar_in)

print(f'encoder output shape: (batch, s, units) {ex_context.shape}')
print(f'input target tokens shape: (batch, t) {ex_tar_in.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

#### Inference

To use it for inference you'll need a couple more methods:

In [ ]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [ ]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [ ]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 
  
  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)
  
  return next_token, done, state

With those extra functions, you can write a generation loop:

In [ ]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

Since the model's untrained, it outputs items from the vocabulary almost uniformly at random.

## The model

Now that you have all the model components, combine them to build the model for training:

In [ ]:
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,
               context_text_processor,
               target_text_processor):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(context_text_processor, units)
    decoder = Decoder(target_text_processor, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, inputs):
    context, x = inputs
    context = self.encoder(context)
    logits = self.decoder(context, x)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

During training the model will be used like this:

In [ ]:
model = Translator(UNITS, context_text_processor, target_text_processor)

logits = model((ex_context_tok, ex_tar_in))

print(f'Context tokens, shape: (batch, s, units) {ex_context_tok.shape}')
print(f'Target tokens, shape: (batch, t) {ex_tar_in.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

### Train

For training, you'll want to implement your own masked loss and accuracy functions:

In [ ]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [ ]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    
    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)
    
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

Configure the model for training:

In [ ]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

The model is randomly initialized, and should give roughly uniform output probabilities. So it's easy to predict what the initial values of the metrics should be:

In [ ]:
vocab_size = 1.0 * target_text_processor.vocabulary_size()

{"expected_loss": tf.math.log(vocab_size).numpy(),
 "expected_acc": 1/vocab_size}

That should roughly match the values returned by running a few steps of evaluation:

In [ ]:
model.evaluate(val_ds, steps=20, return_dict=True)

In [ ]:
history = model.fit(
    train_ds.repeat(), 
    epochs=100,
    steps_per_epoch = 100,
    validation_data=val_ds,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
plt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

### Translate

Now that the model is trained, implement a function to execute the full `text => text` translation. This code is basically identical to the [inference example](#inference) in the [decoder section](#the_decoder), but this also captures the attention weights.

In [ ]:
#@title
@Translator.add_method
def translate(self,
              texts, *,
              max_length=50,
              temperature=0.0):
  # Process the input texts
  context = self.encoder.convert_input(texts)
  batch_size = tf.shape(texts)[0]

  # Setup the loop inputs
  tokens = []
  attention_weights = []
  next_token, done, state = self.decoder.get_initial_state(context)

  for _ in range(max_length):
    # Generate the next token
    next_token, done, state = self.decoder.get_next_token(
        context, next_token, done,  state, temperature)
        
    # Collect the generated tokens
    tokens.append(next_token)
    attention_weights.append(self.decoder.last_attention_weights)
    
    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Stack the lists of tokens and attention weights.
  tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
  self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

  result = self.decoder.tokens_to_text(tokens)
  return result

Here are the two helper methods, used above, to convert tokens to text, and to get the next token:

In [ ]:
# result = model.translate(['ts kh o v r e b i s']) # ts kh o v r e b i s NSG	ts kh o v r e b a
# result = model.translate(['m i z n i t']) # m i z n i t NSI	m i z a n i // *m i z n a
result = model.translate(['sh e s a dz l e b e l i a']) # sh e s a dz l e b e l i a JSN:AUX	sh e s a dz l e b e l i
result[0].numpy().decode()

Use that to generate the attention plot:

In [ ]:
#@title
@Translator.add_method
def plot_attention(self, text, **kwargs):
  assert isinstance(text, str)
  output = self.translate([text], **kwargs)
  output = output[0].numpy().decode()

  attention = self.last_attention_weights[0]

  context = tf_lower_and_split_punct(text)
  context = context.numpy().decode().split()

  output = tf_lower_and_split_punct(output)
  output = output.numpy().decode().split()[1:]

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)

  ax.matshow(attention, cmap='viridis', vmin=0.0)

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + context, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + output, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  ax.set_xlabel('Input text')
  ax.set_ylabel('Output text')

In [ ]:
model.plot_attention('ts kh o v r e b i s') # ts kh o v r e b a 

Translate a few more sentences and plot them:

In [ ]:
%%time
# This is my life.
model.plot_attention('sh e s a dz l e b e l i a')

In [ ]:
%%time
 # Try to find out.'
model.plot_attention('s a sh u a l e b i t')

The short sentences often work well, but if the input is too long the model literally loses focus and stops providing reasonable predictions. There are two main reasons for this:

1. The model was trained with teacher-forcing feeding the correct token at each step, regardless of the model's predictions. The model could be made more robust if it were sometimes fed its own predictions.
2. The model only has access to its previous output through the RNN state. If the RNN state looses track of where it was in the context sequence there's no way for the model to recover. [Transformers](transformer.ipynb) improve on this by letting the decoder look at what it has output so far.

The raw data is sorted by length, so try translating the longest sequence:

In [ ]:
long_text = context_raw[-1]

import textwrap
print('Expected output:\n', '\n'.join(textwrap.wrap(target_raw[-1])))

In [ ]:
model.plot_attention(long_text)

The `translate` function works on batches, so if you have multiple texts to translate you can pass them all at once, which is much more efficient than translating them one at a time:

In [ ]:
inputs = [
    'n e b i s m i e r', # "n e b i s m i e r i"
    'o r g a n o e b i', # "o r g a n o"
    'h kx i t kh a' # "h kx i t kh a v s"
]

In [ ]:
%%time
for t in inputs:
  print(model.translate([t])[0].numpy().decode())

print()

In [ ]:
%%time
result = model.translate(inputs)

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

So overall this text generation function mostly gets the job done, but so you've only used it here in python with eager execution. Let's try to export it next:

### Export

If you want to export this model you'll need to wrap the `translate` method in a `tf.function`. That implementation will get the job done:


In [ ]:
class Export(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
  def translate(self, inputs):
    return self.model.translate(inputs)

In [ ]:
export = Export(model)

Run the `tf.function` once to compile it:

In [ ]:
%%time
_ = export.translate(tf.constant(inputs))

In [ ]:
%%time
result = export.translate(tf.constant(inputs))

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

Now that the function has been traced it can be exported using `saved_model.save`:

In [ ]:
%%time
tf.saved_model.save(export, 'kaLemTranslator',
                    signatures={'serving_default': export.translate})

In [ ]:
!tar cvzf kaLemTranslator.tgz kaLemTranslator/

In [ ]:
# download new model
!wget https://heibox.uni-heidelberg.de/f/9410df17a1004184a37b/?dl=1
!mv index.html?dl=1 kaLemTranslator.tgz
!tar xvzf kaLemTranslator.tgz

In [ ]:
# download old model - different name!!!
!wget https://heibox.uni-heidelberg.de/f/dd94c1eab1564ae0a6fe/?dl=1
!mv index.html?dl=1 translator.tgz
!tar xvzf translator.tgz

In [ ]:
inputs = [
    'n e b i s m i e r', # "n e b i s m i e r i"
    'o r g a n o e b i', # "o r g a n o"
    'h kx i t kh a' # "h kx i t kh a v s"
]

In [ ]:
%%time
reloaded = tf.saved_model.load('kaLemTranslator')
_ = reloaded.translate(tf.constant(inputs)) #warmup

In [ ]:
%%time
result = reloaded.translate(tf.constant(inputs))

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

#### [Optional] Use a dynamic loop

It's worth noting that this initial implementation is not optimal. It uses a python loop:

```
for _ in range(max_length):
  ...
  if tf.executing_eagerly() and tf.reduce_all(done):
    break
```

The python loop is relatively simple but when `tf.function` converts this to a graph, it **statically unrolls** that loop. Unrolling the loop has two disadvantages:

1. It makes `max_length` copies of the loop body. So the generated graphs take longer to build, save and load.
1. You have to choose a fixed value for the `max_length`. 
1. You can't `break` from a statically unrolled loop. The `tf.function`
  version will run the full `max_length` iterations on every call.
  That's why the `break` only works with eager execution. This is
  still marginally faster than eager execution, but not as fast as it could be.


To fix these shortcomings, the `translate_dynamic` method, below, uses a tensorflow loop:

```
for t in tf.range(max_length):
  ...
  if tf.reduce_all(done):
      break
```

It looks like a python loop, but when you use a tensor as the input to a `for` loop (or the condition of a `while` loop) `tf.function` converts it to a dynamic loop using operations like `tf.while_loop`. 

There's no need for a `max_length` here it's just in case the model gets stuck generating a loop like: `the united states of the united states of the united states...`.

On the down side, to accumulate tokens from this dynamic loop you can't just append them to a python `list`, you need to use a `tf.TensorArray`:

```
tokens = tf.TensorArray(tf.int64, size=1, dynamic_size=True)
...
for t in tf.range(max_length):
    ...
    tokens = tokens.write(t, next_token) # next_token shape is (batch, 1)
  ...
  tokens = tokens.stack()
  tokens = einops.rearrange(tokens, 't batch 1 -> batch t')
```

This version of the code can be quite a bit more efficient:

In [ ]:
#@title
@Translator.add_method
def translate(self,
              texts,
              *,
              max_length=500,
              temperature=tf.constant(0.0)):
  shape_checker = ShapeChecker()
  context = self.encoder.convert_input(texts)
  batch_size = tf.shape(context)[0]
  shape_checker(context, 'batch s units')

  next_token, done, state = self.decoder.get_initial_state(context)

  # initialize the accumulator
  tokens = tf.TensorArray(tf.int64, size=1, dynamic_size=True)

  for t in tf.range(max_length):
    # Generate the next token
    next_token, done, state = self.decoder.get_next_token(
        context, next_token, done, state, temperature)
    shape_checker(next_token, 'batch t1')

    # Collect the generated tokens
    tokens = tokens.write(t, next_token)

    # if all the sequences are done, break
    if tf.reduce_all(done):
      break

  # Convert the list of generated token ids to a list of strings.
  tokens = tokens.stack()
  shape_checker(tokens, 't batch t1')
  tokens = einops.rearrange(tokens, 't batch 1 -> batch t')
  shape_checker(tokens, 'batch t')

  text = self.decoder.tokens_to_text(tokens)
  shape_checker(text, 'batch')

  return text

With eager execution this implementation performs on par with the original:

In [ ]:
%%time
result = model.translate(inputs)

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

But when you wrap it in a `tf.function` you'll notice two differences.

In [ ]:
class Export(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
  def translate(self, inputs):
    return self.model.translate(inputs)

In [ ]:
export = Export(model)

First, it's much quicker to trace, since it only creates one copy of the loop body:

In [ ]:
%%time
_ = export.translate(inputs)

The `tf.function` is much faster than running with eager execution, and on small inputs it's often several times faster than the unrolled version, because it can break out of the loop.

In [ ]:
%%time
result = export.translate(inputs)

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

So save this version as well:

In [ ]:
%%time
tf.saved_model.save(export, 'kaLemDynamic_translator',
                    signatures={'serving_default': export.translate})

In [ ]:
!tar cvzf kaLemDynamic_translator.tgz kaLemDynamic_translator/

In [ ]:
# download new model
!wget https://heibox.uni-heidelberg.de/f/ecce3b430f1547cd8d7d/?dl=1
!mv index.html?dl=1 kaLemDynamic_translator.tgz
!tar xvzf kaLemDynamic_translator.tgz


In [ ]:
# download old model - name changed!
!wget https://heibox.uni-heidelberg.de/f/f06b1fb72d4347319e2a/?dl=1
!mv index.html?dl=1 dynamic_translator.tgz
!tar xvzf dynamic_translator.tgz

In [ ]:
# !tar xvzf dynamic_translator.tgz

In [ ]:
### if needed -- if runtime restarted
!pip install "tensorflow-text>=2.11"
!pip install einops

In [19]:
### if needed -- if runtime restarted
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text


inputs = [
    'n e b i s m i e r', # "n e b i s m i e r i"
    'o r g a n o e b i', # "o r g a n o"
    'h kx i t kh a' # "h kx i t kh a v s"
]

In [ ]:
inputs = [
    'n e b i s m i e r', # "n e b i s m i e r i"
    'o r g a n o e b i', # "o r g a n o"
    'h kx i t kh a' # "h kx i t kh a v s"
]

In [20]:
%%time
reloaded = tf.saved_model.load('kaLemDynamic_translator')
_ = reloaded.translate(tf.constant(inputs)) #warmup

CPU times: user 9.97 s, sys: 800 ms, total: 10.8 s
Wall time: 11.1 s


In [21]:
%%time
result = reloaded.translate(tf.constant(inputs))

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

n e b i s m i e r i 
o r g a n o     
h kx i t kh e b a   

CPU times: user 35.9 ms, sys: 1.85 ms, total: 37.8 ms
Wall time: 44.5 ms


## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs.
* Try the [transformer tutorial](transformer.ipynb) which implements a similar translation task but uses transformer layers instead of RNNs. This version also uses a `text.BertTokenizer` to implement word-piece tokenization.
* Visit the [`tensorflow_addons.seq2seq` tutorial](https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt), which demonstrates a higher-level functionality for implementing this sort of sequence-to-sequence model, such as `seq2seq.BeamSearchDecoder`.

## Evaluation experiment

- preparing evaluation set
- generating translation
- comparing to the silver standard (TreeTagger output)



In [22]:
def readEvalSet2list(SFIn):
    LEval = []
    LSilverStandard = []

    with open(SFIn, 'r') as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip('\n')
            try:
                SWord, SLemSilver = SLine.split('\t')
                LEval.append(SWord)
                LSilverStandard.append(SLemSilver)
            except:
                continue

    return LEval, LSilverStandard


In [ ]:
LEval, LSilverStandard = readEvalSet2list('/content/georgianrandom02-ka-change-chars.txt_test.txt')
print(len(LEval))
print(len(LSilverStandard))

In [ ]:
result6k = reloaded.translate(tf.constant(LEval))


In [ ]:
LResults6k = []
for el in result6k:
    SLemResDecoded = el.numpy().decode()
    SLemResDecoded = SLemResDecoded.rstrip(' ')
    LResults6k.append(SLemResDecoded)
print(len(result6k))
print(len(LResults6k))

In [ ]:
def evaluateResult(LInput, LResult, LSilverStandard):
    LCorrect = []
    LWrong = []
    LTotal = []
    for index in range(len(LResult)):
        WInput = LInput[index]
        WResult = LResult[index]
        WSilverStandard = LSilverStandard[index]
        if WResult == WSilverStandard:
            LCorrect.append((WInput, WResult, WSilverStandard))
        else:
            LWrong.append((WInput, WResult, WSilverStandard))
        LTotal.append((WInput, WResult, WSilverStandard))

    return LCorrect, LWrong, LTotal

In [ ]:
LCorrect, LWrong, LTotal = evaluateResult(LEval, LResults6k, LSilverStandard)
ICorrect = len(LCorrect)
IWrong = len(LWrong)
ITotal = len(LTotal)

print(f'c {ICorrect}, w {IWrong}, t {ITotal}')
print(f'pcC:{ICorrect/ITotal}')
print(f'pcW:{IWrong/ITotal}')


In [ ]:
def readTableWSReverse(SFIn):
    DMap = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip('\n')
            try:
                KA, LA = SLine.split('\t')
                DMap[LA] = KA
            except:
                continue
    print(len(DMap))
    return DMap

In [ ]:
DLat2Geo = readTableWSReverse('/content/ltKA.txt')


In [ ]:
def transliterateString(String, DMapChars, BMerge = True):
    SOut = ''
    LChars = String.split(' ')
    LCharsOut = []
    for SChar in LChars:
        try:
          SCharOut = DMapChars[SChar]
        except:
          SCharOut = SChar
        LCharsOut.append(SCharOut)
    if BMerge:
        SOut = ''.join(LCharsOut)
    else:
        SOut = ' '.join(LCharsOut)
    return SOut

In [ ]:
def print2TupleList(T2Tuple, SFOut, DTransliterationLAT2GEO = None):
    FOut = open(SFOut, 'w')
    for S1, S2, S3 in T2Tuple:
        if DTransliterationLAT2GEO:
            S1 = transliterateString(S1, DTransliterationLAT2GEO)
            S2 = transliterateString(S2, DTransliterationLAT2GEO)
            S3 = transliterateString(S3, DTransliterationLAT2GEO)

        FOut.write(f'{S1}\t{S2}\t{S3}\n')
    FOut.flush()

In [ ]:
print2TupleList(LTotal, 'eval--total-list.txt', DLat2Geo)
print2TupleList(LCorrect, 'eval--correct-list.txt', DLat2Geo)
print2TupleList(LWrong, 'eval--wrong-list.txt', DLat2Geo)

## todo:
- evaluate for each part-of-speech code
- evaluate for first letter in each part-of-speech code
- evaluate for different frequency ranges
- compare with quasi-inflection based approach
- compare affixes that were successfully-unsuccessfully changed
- develop a hybrid architecture for lemmatization

### further steps
- train the model with the part-of-speech information integrated
- use the results from evaluation to maximise the chance of correct lemmatization given the part-of-speech code and available approaches


# Evaluation on unknown words from specialized corpora



In [ ]:
!head -n 10 /content/specialised-corpora-s02-vert/cFiktion.vert

### selecting unknown

In [24]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cFiktion.vert >/content/specialised-corpora-s02-vert/cFiktion-unknown.vert
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cNaturwissenschaft.vert >/content/specialised-corpora-s02-vert/cNaturwissenschaft-unknown.vert
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cRechtswissenschaft.vert >/content/specialised-corpora-s02-vert/cRechtswissenschaft-unknown.vert


135122
128706
194695


### selecting words with only letters

In [25]:
# !awk -F '\t' '($3 ~ /[Ⴀ-ჼ]/){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-ka-alphabet.txt
!awk -F '\t' '($1 ~ /[Ⴀ-ჼ]/){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cFiktion-unknown.vert >/content/specialised-corpora-s02-vert/cFiktion-u2alph.vert
!awk -F '\t' '($1 ~ /[Ⴀ-ჼ]/){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cNaturwissenschaft-unknown.vert >/content/specialised-corpora-s02-vert/cNaturwissenschaft-u2alph.vert
!awk -F '\t' '($1 ~ /[Ⴀ-ჼ]/){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' </content/specialised-corpora-s02-vert/cRechtswissenschaft-unknown.vert >/content/specialised-corpora-s02-vert/cRechtswissenschaft-u2alph.vert



127210
122634
174800


### print frq dictionary files



In [1]:
### if needed -- 
def corp2frqDict(SFIn):
    DFrq = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip()
            # LLine = SLine.split('\t')
            try:
                DFrq[SLine]+=1
            except:
                DFrq[SLine]=1
    return DFrq


def printFrqDict(DFrq, SFOut):
    FOut = open(SFOut, 'w')
    count = 0
    for key, val in sorted(DFrq.items(), key=lambda item: item[1], reverse=True):
        count+=1
        FOut.write(f'{count}\t{key}\t{val}\n')
    FOut.flush()

In [2]:
DFrqKnown = corp2frqDict('/content/specialised-corpora-s02-vert/cFiktion-u2alph.vert')
print(len(DFrqKnown))
printFrqDict(DFrqKnown, '/content/specialised-corpora-s02-vert/cFiktion-u3frq.vert.txt')

DFrqKnown = corp2frqDict('/content/specialised-corpora-s02-vert/cNaturwissenschaft-u2alph.vert')
print(len(DFrqKnown))
printFrqDict(DFrqKnown, '/content/specialised-corpora-s02-vert/cNaturwissenschaft-u3frq.vert.txt')

DFrqKnown = corp2frqDict('/content/specialised-corpora-s02-vert/cRechtswissenschaft-u2alph.vert')
print(len(DFrqKnown))
printFrqDict(DFrqKnown, '/content/specialised-corpora-s02-vert/cRechtswissenschaft-u3frq.vert.txt')

58605
53277
34218


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/70af69f479044d228e6d/?dl=1
!mv index.html?dl=1 ltKA.txt

In [4]:
def readTableWS(SFIn):
    DMap = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip('\n')
            try:
                KA, LA = SLine.split('\t')
                DMap[KA] = LA
            except:
                continue
    print(len(DMap))
    return DMap

DTransliterationKA = readTableWS('/content/ltKA.txt')

34


In [3]:
S2Print = ''.join([DTransliterationKA[x] for x in 'ქართული' if x in DTransliterationKA])
print(S2Print)

kartuli


In [5]:
### run this - new function!
import os, sys, re
def splitCharsUnknown(SFIn, SFOut, BSplitChars = True, RandomEvery = 0, DTranslit = None, Lemmatize = False):
    LSUnknownWords = []
    FOut = open(SFOut, 'w')
    if RandomEvery:
        FOutTest = open(SFOut + '_test.txt', 'w')
    
    with open(SFIn) as FIn:
        countW = 0
        for SLine in FIn:
            countW += 1
            SLine = SLine.rstrip()
            LLine = SLine.split('\t')
            try:
                SRank = LLine[0]
                SWord = LLine[1]
                SPoS = LLine[2]
                SLem = LLine[3]
                SFrq = LLine[4]
            except:
                SRank = ''
                SWord = ''
                SPoS = ''
                SLem = ''
                SFrq = ''
                sys.stderr.write('.e')
                continue

            if BSplitChars:
                SWord = ' '.join(list(SWord))
                # RHS = ' '.join(list(RHS))


            if DTranslit:
                SWord = ''.join([DTransliterationKA[x] for x in SWord if x in DTranslit])
                # RHS = ''.join([DTransliterationKA[x] for x in RHS if x in DTranslit])
                

            if RandomEvery and countW % RandomEvery == 0:
                FOutTest.write(f'{SRank}\t{SWord}\t{SPoS}\t{SLem}\t{SFrq}\n')
                continue
            
            LSUnknownWords.append(SWord)
            FOut.write(f'{SRank}\t{SWord}\t{SPoS}\t{SLem}\t{SFrq}\n')

    return LSUnknownWords

In [6]:
LSUnknownWordsF = splitCharsUnknown('/content/specialised-corpora-s02-vert/cFiktion-u3frq.vert.txt', '/content/specialised-corpora-s02-vert/cFiktion-u4frqTS.vert.txt', BSplitChars = True, DTranslit = DTransliterationKA)
LSUnknownWordsN = splitCharsUnknown('/content/specialised-corpora-s02-vert/cNaturwissenschaft-u3frq.vert.txt', '/content/specialised-corpora-s02-vert/cNaturwissenschaft-u4frqTS.vert.txt', BSplitChars = True, DTranslit = DTransliterationKA)
LSUnknownWordsR = splitCharsUnknown('/content/specialised-corpora-s02-vert/cRechtswissenschaft-u3frq.vert.txt', '/content/specialised-corpora-s02-vert/cRechtswissenschaft-u4frqTS.vert.txt', BSplitChars = True, DTranslit = DTransliterationKA)

In [7]:
print(len(LSUnknownWordsF))
print(len(LSUnknownWordsN))
print(len(LSUnknownWordsR))


58605
53277
34218


In [ ]:
# 3
### if needed -- if runtime restarted
!pip install "tensorflow-text>=2.11"
!pip install einops

In [8]:
# 3
### if needed -- if runtime restarted
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text


inputs = [
    'n e b i s m i e r', # "n e b i s m i e r i"
    'o r g a n o e b i', # "o r g a n o"
    'h kx i t kh a' # "h kx i t kh a v s"
]

In [9]:
# 3
%%time
reloaded = tf.saved_model.load('kaLemDynamic_translator')
_ = reloaded.translate(tf.constant(inputs)) #warmup

CPU times: user 10.2 s, sys: 373 ms, total: 10.6 s
Wall time: 11.3 s


In [10]:
print(LSUnknownWordsF)

['t o m a s', 'b u d e n b r o kx e b i', 'm a n i', 'kx', 'kx   m', 'kx  m', 'p r a n ts', 'sh tx e p a n', '', 'ts v a i g i', 'kx   s', 'kx  s', 'v a n', 'p r o i l a i n', 'kx a p kx a', 'tx o n i m', 'kx o n s u l m a', 'tx o m', 'h a n o', 'kx a r l', 'b u d e n b r o kx s', 'kx o n s u l i', 'g e r d a', 'p r i d a', 'h a n o s', 'b a r n a b a s', 'g r i u n l i h i', 'px e r m a n e d e r m a', 'sh i l e r i', 'kx l a m i s', 'm tx kx i ts e d', 'm a l i n a s', 's e n a tx o r m a', 'i o h a n', 't o m a s s', 'i v a n s', 'b a r n a b a', 'd a n i e l', 'k r i s tx i a n s', 'g a i m e o r a', 'p o n', 'e d i t i', 's a qx v a r e l o', 'm a r g o', 's a m a g i e r o d', 'b u d e n b r o kx e b i s', 'm o o r', 't o m a s i', 'g o d o l sh i', 'kx o n d o r i', 'm kh a tx v a r m a', 'm o o r', 'm z e r a s', 'v e k i l m a', 'i d a', 'u t u o d', 'sh v a i ts e r', 'n a z a d', 'k r i s tx i a n m a', 'l e i tx e n a n tx o', 'kx o n d o r m a', 'g r i u n l i h m a', 's e

In [13]:
# resultUnknownF0 = reloaded.translate(tf.constant(LSUnknownWordsF[:20000]))
# resultUnknownN0 = reloaded.translate(tf.constant(LSUnknownWordsN[:2000]))
# resultUnknownN1 = reloaded.translate(tf.constant(LSUnknownWordsN[2000:4000]))
# resultUnknownN2 = reloaded.translate(tf.constant(LSUnknownWordsN[4000:6000]))
# resultUnknownN3 = reloaded.translate(tf.constant(LSUnknownWordsN[6000:8000]))
# resultUnknownN4 = reloaded.translate(tf.constant(LSUnknownWordsN[8000:10000]))
# resultUnknownN5 = reloaded.translate(tf.constant(LSUnknownWordsN[10000:12000]))
# resultUnknownN6 = reloaded.translate(tf.constant(LSUnknownWordsN[12000:14000]))
# resultUnknownN7 = reloaded.translate(tf.constant(LSUnknownWordsN[14000:16000]))
#### resultUnknownN8 = reloaded.translate(tf.constant(LSUnknownWordsN[16000:18000]))
resultUnknownN80 = reloaded.translate(tf.constant(LSUnknownWordsN[16000:16200]))
resultUnknownN81 = reloaded.translate(tf.constant(LSUnknownWordsN[16200:16400]))
resultUnknownN82 = reloaded.translate(tf.constant(LSUnknownWordsN[16400:16600]))
resultUnknownN83 = reloaded.translate(tf.constant(LSUnknownWordsN[16600:16800]))
resultUnknownN84 = reloaded.translate(tf.constant(LSUnknownWordsN[16800:17000]))
resultUnknownN85 = reloaded.translate(tf.constant(LSUnknownWordsN[17000:17200]))
resultUnknownN86 = reloaded.translate(tf.constant(LSUnknownWordsN[17200:17400]))
resultUnknownN87 = reloaded.translate(tf.constant(LSUnknownWordsN[17400:17600]))
resultUnknownN88 = reloaded.translate(tf.constant(LSUnknownWordsN[17600:17800]))
resultUnknownN89 = reloaded.translate(tf.constant(LSUnknownWordsN[17800:18000]))
## resultUnknownN8 = reloaded.translate(tf.constant(LSUnknownWordsN[16000:18000]))

## resultUnknownN9 = reloaded.translate(tf.constant(LSUnknownWordsN[18000:20000])) # 3 min
# resultUnknownN10 = reloaded.translate(tf.constant(LSUnknownWordsN[:2000]))
# resultUnknownR0 = reloaded.translate(tf.constant(LSUnknownWordsR[:20000]))

In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def translate_list(list, itemsPerListAtOnce = 200):
    LResultsUnknown = []
    chunkCount = 0
    LLWords2Translate = chunks(list, itemsPerListAtOnce)
    for LWords2Trans in LLWords2Translate:
        chunkCount += 1
        print(chunkCount + ' :start')
        LResultsUnkTemp = reloaded.translate(tf.constant(LWords2Trans))
        LResultsUnknown.extend(LResultsUnkTemp)
        print(chunkCount + ' :done')
    return LResultsUnknown


In [ ]:
LResultsUnknownF = translate_list(LSUnknownWordsF)

In [43]:
def decodeResults(LResults0):
    LResults1 = []
    for el in LResults0:
        SLemResDecoded = el.numpy().decode()
        SLemResDecoded = SLemResDecoded.rstrip(' ')
        LResults1.append(SLemResDecoded)
    print(len(LResults0))
    print(len(LResults1))
    return LResults1

In [44]:
resultUnknownF = decodeResults(resultUnknownF0)
resultUnknownN = decodeResults(resultUnknownN0)
resultUnknownR = decodeResults(resultUnknownR0)

10000
10000
10000
10000
10000
10000


In [45]:
def lists2dict(list1, list2):
    Di = {}
    for i in range(len(list1)):
        lhs = list1[i]
        rhs = list2[i]

        Di[lhs] = rhs

    return Di

In [46]:
DiF = lists2dict(LSUnknownWordsF[:5000], resultUnknownF)
DiN = lists2dict(LSUnknownWordsF[:5000], resultUnknownN)
DiR = lists2dict(LSUnknownWordsF[:5000], resultUnknownR)

In [47]:
# run if needed
def readTableWSReverse(SFIn):
    DMap = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip('\n')
            try:
                KA, LA = SLine.split('\t')
                DMap[LA] = KA
            except:
                continue
    print(len(DMap))
    return DMap

def transliterateString(String, DMapChars, BMerge = True):
    SOut = ''
    LChars = String.split(' ')
    LCharsOut = []
    for SChar in LChars:
        try:
          SCharOut = DMapChars[SChar]
        except:
          SCharOut = SChar
        LCharsOut.append(SCharOut)
    if BMerge:
        SOut = ''.join(LCharsOut)
    else:
        SOut = ' '.join(LCharsOut)
    return SOut

In [48]:

DLat2Geo = readTableWSReverse('/content/ltKA.txt')

34


In [49]:
def printDict(DFrq, SFOut, Transliterate = True):
    FOut = open(SFOut, 'w')
    count = 0
    for key, val in sorted(DFrq.items(), key=lambda item: item[1], reverse=True):
        if Transliterate:
            key = transliterateString(key, DLat2Geo)
            val = transliterateString(key, DLat2Geo)
        count+=1
        FOut.write(f'{count}\t{key}\t{val}\n')
    FOut.flush()

In [50]:
printDict(DiF, 'evalF.txt')
printDict(DiN, 'evalN.txt')
printDict(DiR, 'evalR.txt')